<h1>Create React App Structure Using Multi Agents </h1>
<h3>Use OpenAI and deepseek to create an app structure for React app. </h3>

In [ ]:
# Let's import environment variables
from dotenv import load_dotenv
load_dotenv(override=True)

In [ ]:
import os
import json
from typing import Dict, Any
from IPython.display import Markdown, display
from openai import OpenAI

In [ ]:
openai_api_key = os.getenv('OPENAI_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if not openai_api_key:
    print('Missing OpenaAI API key.')
if not deepseek_api_key:
    print('Missing Deepseek API key')
if openai_api_key and deepseek_api_key:
    print(f'OpenAI: {openai_api_key[-10:]}\n')
    print(f'Deepseek: {deepseek_api_key[-10:]}\n')


In [ ]:
app = {"app_name": "Small Business Idea"}

In [ ]:
openai = OpenAI()
deepseek = OpenAI(api_key=deepseek_api_key, 
    base_url="https://api.deepseek.com")

In [ ]:
# system prompt and user prompt  
 
system_prompt = """
You're an entrepreneur focused on developing and investing in 
emerging AI-driven SaaS applications that solve critical pain
points for small businesses—such as bookkeeping, reservations,
tax preparation, and employee records management. 

You prioritize solutions leveraging agentic AI to address 
real-world business challenges with minimal human oversight,
delivering both scalability and innovation. Your goal is to 
identify ideas with the highest potential for market disruption
while helping small businesses save time and money.

List all the business areas that might be worth exploring for 
Agentic AI.

"""

user_prompt = "List all the business area that might be worth exploring for Agentic AI."

messages = [
    {"role": "system", "content":system_prompt},
    {"role": "user", "content": user_prompt},
]

# Call openai
response = deepseek.chat.completions.create(
    model="deepseek-chat",
    messages=messages
)

business_ideas = response.choices[0].message.content
display(Markdown(business_ideas))

In [ ]:
# Best idea prompt
selected_idea_prompt = f"""Select the best idea from the list: {business_ideas} areas. 
Give reasons and why this pain point is the best to solve.
List only the top idea."""

second_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": selected_idea_prompt}
]

In [ ]:
# Call openai to select the best idea 
response = openai.chat.completions.create(
    messages=second_messages,
    model="gpt-4.1-mini"
)

selected_idea = response.choices[0].message.content
display(Markdown(selected_idea))

In [ ]:
# Add idea and pain points 
app['idea'] = selected_idea

In [ ]:
# Let's create an app structure for the selected idea 
# Break the f-string into smaller parts for better readability and to avoid nesting issues
system_prompt = "Please create a react app file directory structure. You're given the business idea, along with the following pain points."
structure_prompt = """
Respond in clear JSON format only, remove any backticks, extra spaces. The structure should also include 
frontend pages, authentication, api, stripe payment, and a backend database along with
any necessary directories and files for the app to work without any errors.
Respond with JSON format with name of the file, and path where the file should be stored, for example:

{
  "root": {
    "public": {
      "index.html": "root/public/index.html",
      "css": {
        "style.css": "root/public/css/style.css"
      },
      "images": {
        "logo.png": "root/public/images/logo.png"
      }
    }
  }
}
"""

create_structure_prompt = f"{system_prompt}\n{structure_prompt}"

structure_prompt= [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": create_structure_prompt}
]

response  = openai.chat.completions.create(
    messages=structure_prompt,
    model="gpt-4.1-mini"    
)
structure = response.choices[0].message.content
display(Markdown(structure))


In [ ]:
app["app_structure"] =  structure

In [ ]:
structure_check_prompt = f"""You're a an expert react app developer. You validate 
react app file structure for the idea 
{selected_idea}\n.
If there're any errors with the structure, for example if there're missing files, directories, or any extra 
modifications needed to make the structure better, please respond 
with 'Needs modification' text/word only. 

If the structure doesn't need modification, simply 
respond with 'Correct structure' text/word only.
"""

structure_check= [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": structure_check_prompt}
]

In [ ]:
"""
We need to double check if the app structure is correct. We can use other models, 
deepseek seems to add extra files, and stays out of context, so let's stick with 
openai for now. 
"""
response = deepseek.chat.completions.create(
    messages=structure_check,
    model="deepseek-chat"  
)

double_check = response.choices[0].message.content
display(Markdown(double_check))

In [ ]:
# Check if the file structure is correct 
correct_structure = (double_check == 'Correct structure')

if not correct_structure: # Only try if structure is incorrect 
    print(f"Structure needs correction: {double_check}")
    max_count = 0
    updated_structure = structure # Start with the original 
    
    while max_count < 3 and not correct_structure:
        
        content = f"""Please correct the file structure {structure} for the selected idea 
        {selected_idea}. Respond with clear JSON format only, with no backticks."""
        json_format = f"""Please follow this example JSON structure:
        If the structure is correct please respond with only 'Correct structure' text only."""
        example ="""
        {
            "root": {
                "public": {
                "index.html": "root/public/index.html",
                "css": {
                    "style.css": "root/public/css/style.css"
                },
                "images": {
                    "logo.png": "root/public/images/logo.png"
                }
                }
            }
        }
        """
        
        retry_message = f"{content}\n {selected_idea}\n{json_format}\n{example}"
        
        response = openai.chat.completions.create(
        messages=[
            {"role":"system", "content": system_prompt},
            {"role": "user","content": f"{retry_message}"}
        ],
        model="gpt-4.1-mini"
        )
        
        response = response.choices[0].message.content
        
        if response == 'Correct structure':
            correct_structure = True
            print("Structure is already correct, no modification needed.")
            
        else:
            # Retry
            updated_structure = response 
            max_count += 1 
            print(f">>> Retrying...{max_count}")
    
    # Update the app structure with the last/corrected version
    app['app_structure'] = json.loads(updated_structure )
    
else:
    print("Structure is already correct")
    app["app_structure"] = json.loads(structure)

In [ ]:
app['app_structure']

In [ ]:
# Save as JSON file 
with open('app_structure.json', 'w') as f:
    json.dump(app['app_structure'],f, indent=4)
    
    print("App structure saved to app_structure.json")

In [ ]:
# Create the file structure recursively, from structure in current directory
def create_file_structure(structure: Dict, parent_dir:str='.'):
    """Create file structure recursively from structure. """
    try:
        for file, folder in structure.items():
            path = os.path.join(parent_dir, file)
            if isinstance(folder, dict):
                # It's a directory
                os.makedirs(path, exist_ok=True)
                create_file_structure(folder, path) # recursively create the sub folder structure
            else:
                # It's a file, create empty file
                os.makedirs(parent_dir, exist_ok=True)
                
                # Check file extension
                valid_extensions = ('.ts', '.tsx', '.md', '.js', '.css', '.json', '.jsx', '.html', '.txt', '.db', '.py', '.sql')
                
                if file.endswith(valid_extensions):
                    with open(path, 'w') as f:
                        pass  # Create an empty file
                else:
                    print(f'Unknown file type {file}')

    except Exception as e:
        print(f"Error creating file structure: {e}")
    

In [ ]:
# Open the app_structure file 
filepath = os.path.join(os.getcwd(),'app_structure.json')

with open(filepath, 'r', encoding='utf-8') as f:
    app_structure = json.load(f) 

create_file_structure(app_structure, parent_dir='./app/')


In [ ]:
system_prompt = f"""You're Senior react developer with over 10 years of experience. 
"""
user_prompt = f"""You're given the following app details in the {app['app_structure']}\n
for the {selected_idea}. Please write the following files . 

"package.json": "root/package.json"
"README.md": "root/README.md"
".gitignore": "root/.gitignore"
"webpack.config.js": "root/webpack.config.js"
"""

In [ ]:
messages = [
    {"role":"system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

response = openai.chat.completions.create(
    messages=messages,
    model="gpt-4.1-mini"
)

source_response = response.choices[0].message.content
display(Markdown(source_response))
